# Merge sensor data with MET lables

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np


os.chdir("/home/cernerrood246/University/DataMining")
pd.set_option("display.precision", 2)

In [4]:
participants_list = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12', 'P13', 'P14', 'P15', 'P16', 'P17']

## Load MET

In [17]:
MET_all_list = [pd.read_pickle(f"preprocessed_dataset/{participant}/MET.pkl") for participant in participants_list]
MET_all_list[0]


,Time[s],Delta_Time,MET
Time,,,
2021-12-03 16:58:51,1.0,0 days 00:00:00,1.31
2021-12-03 16:58:52,2.0,0 days 00:00:01,1.31
2021-12-03 16:58:53,3.0,0 days 00:00:02,1.52
2021-12-03 16:58:54,4.0,0 days 00:00:03,1.63
2021-12-03 16:58:55,5.0,0 days 00:00:04,1.56
...,...,...,...
2021-12-03 17:28:48,1798.0,0 days 00:29:57,5.32
2021-12-03 17:28:49,1799.0,0 days 00:29:58,5.11
2021-12-03 17:28:50,1800.0,0 days 00:29:59,4.46
